In [1]:
import pandas as pd
import numpy as np

In [2]:
cars_data=pd.read_csv(r"C:\Python Imarticus\Basics\Decision Tree\cars.csv",header=None)
cars_data.head()

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [3]:
print(cars_data.shape)
print()
print(cars_data.info())

(1728, 7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1728 non-null   object
 1   1       1728 non-null   object
 2   2       1728 non-null   object
 3   3       1728 non-null   object
 4   4       1728 non-null   object
 5   5       1728 non-null   object
 6   6       1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB
None


In [4]:
cars_data.columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot','safety', 'classes']
cars_data.head()

,buying,maint,doors,persons,lug_boot,safety,classes
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


### Pre Processing Data

In [5]:
#Missing Values

cars_data.isnull().sum()

buying      0
maint       0
doors       0
persons     0
lug_boot    0
safety      0
classes     0
dtype: int64

In [6]:
#optional to create copy of Df(general practice in industry)

cars_df=pd.DataFrame.copy(cars_data)

In [7]:
colname=[]
for x in cars_df.columns:
    if cars_df[x].dtype=='object':
        colname.append(x)

from sklearn import preprocessing
le=preprocessing.LabelEncoder()
for x in colname:
    cars_df[x]=le.fit_transform(cars_df[x])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    print('Feature', x)
    print('mapping', le_name_mapping)

Feature buying
mapping {'high': 0, 'low': 1, 'med': 2, 'vhigh': 3}
Feature maint
mapping {'high': 0, 'low': 1, 'med': 2, 'vhigh': 3}
Feature doors
mapping {'2': 0, '3': 1, '4': 2, '5more': 3}
Feature persons
mapping {'2': 0, '4': 1, 'more': 2}
Feature lug_boot
mapping {'big': 0, 'med': 1, 'small': 2}
Feature safety
mapping {'high': 0, 'low': 1, 'med': 2}
Feature classes
mapping {'acc': 0, 'good': 1, 'unacc': 2, 'vgood': 3}


In [8]:
cars_df.head()

,buying,maint,doors,persons,lug_boot,safety,classes
0,3,3,0,0,2,1,2
1,3,3,0,0,2,2,2
2,3,3,0,0,2,0,2
3,3,3,0,0,1,1,2
4,3,3,0,0,1,2,2


In [12]:
cars_df.classes.value_counts()

2    1210
0     384
1      69
3      65
Name: classes, dtype: int64

In [9]:
#Splitting data into ind and dep var

X=cars_df.values[:,:-1]
Y=cars_df.values[:,-1]
Y=Y.astype(int)

In [10]:
#Scaling data: Optional here as we have almost we have same range values

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [11]:
#Splitting data

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3,random_state=10)

### Model1: Base DT

In [22]:
'''criterion=gini is the default for decision tree classifier
   random_state, lets assume there are 2 vars which have same gini index it selects randomly when we run multiple times 
               the script later, in oder to select the same var every time random_state is passed'''

from sklearn.tree import DecisionTreeClassifier
model_DecisionTree=DecisionTreeClassifier(criterion='gini',random_state=10)
model_DecisionTree.fit(X_train,Y_train)
Y_pred=model_DecisionTree.predict(X_test)

In [23]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cfm=confusion_matrix(Y_test,Y_pred)
print(cfm)

print()

print("classification report:")
print(classification_report(Y_test,Y_pred))

print()

acc=accuracy_score(Y_test, Y_pred)
print("Accuracy of the model: ",acc)

[[101   0   1   0]
 [  2  19   0   0]
 [  0   0 371   0]
 [  1   0   0  24]]

classification report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       102
           1       1.00      0.90      0.95        21
           2       1.00      1.00      1.00       371
           3       1.00      0.96      0.98        25

    accuracy                           0.99       519
   macro avg       0.99      0.96      0.98       519
weighted avg       0.99      0.99      0.99       519


Accuracy of the model:  0.9922928709055877


In [24]:
'''Decision Tree can also be used for feature selection during EDA. Based on the imp of features we select the imp 
ones and proceed for building model.sum of the importance values of all vars=1, the highest value indicates the most imp var'''

print(list(zip(cars_df.columns,model_DecisionTree.feature_importances_)))

[('buying', 0.1510848831946676), ('maint', 0.2506508516803624), ('doors', 0.060026331736828115), ('persons', 0.19355707150872045), ('lug_boot', 0.09892620952419463), ('safety', 0.2457546523552268)]


In [ ]:
'''By above we see doors is the least importance but we didnt get kind of zero value. There will be some datasets where we have 
multiple vars we get much lower values. We can go ahead and ignore them considering the domain knowledge'''

#### Plot DT

In [25]:
'''export_graphviz : exports into text file, the step by step procedure followed while building DT 
   feature_names: prints the name of the vars, if not used tree is plotted with indices.
   
   File is generated in the specified directory now upload the code in webgraphviz.com to plot the decision tree'''
    
from sklearn import tree
with open("C:\Python Imarticus\Basics\Plots_Reports\cars_model_DecisionTree.txt", "w") as f:
    f = tree.export_graphviz(model_DecisionTree, feature_names=cars_df.columns[:-1],out_file=f)

### Model2: Tuning DT by Pruning Method(Perform this step when necessary)

Ideally above model is performing better on test data. But if we think that this model is overfitted then we prune the tree
for tuning the base model. Change in hyperparameters is done for pruning

In [26]:
'''min_samples_leaf: by defualt is 1 which means every leaf has 1 obs which means it is overfiited, to avoid this we give min 
                     no of obs
   max_depth: total levels of the tree,ideally it grows to full levels, to aviod we give certain no(generally 2/3rd of the 
            base model value-model_DecisionTree.get_depth()) after which there is no further splitting'''

from sklearn.tree import DecisionTreeClassifier
model_DecisionTree=DecisionTreeClassifier(criterion='gini',min_samples_leaf=5,max_depth=10,random_state=10)
model_DecisionTree.fit(X_train,Y_train)
Y_pred=model_DecisionTree.predict(X_test)

In [27]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cfm=confusion_matrix(Y_test,Y_pred)
print(cfm)

print()

print("classification report:")
print(classification_report(Y_test,Y_pred))

print()

acc=accuracy_score(Y_test, Y_pred)
print("Accuracy of the model: ",acc)

[[ 92   6   4   0]
 [  0  17   2   2]
 [  8   0 363   0]
 [  3   5   0  17]]

classification report:
              precision    recall  f1-score   support

           0       0.89      0.90      0.90       102
           1       0.61      0.81      0.69        21
           2       0.98      0.98      0.98       371
           3       0.89      0.68      0.77        25

    accuracy                           0.94       519
   macro avg       0.84      0.84      0.84       519
weighted avg       0.95      0.94      0.94       519


Accuracy of the model:  0.9421965317919075


In [28]:
#Plot Pruned DT

from sklearn import tree
with open("C:\Python Imarticus\Basics\Plots_Reports\cars_pruned_model_DecisionTree.txt", "w") as f:
    f = tree.export_graphviz(model_DecisionTree, feature_names=cars_df.columns[:-1],out_file=f)

In [ ]:
'''By above we see lower accuracy. Original model was doing good job. This shows we shouldn't do unnecessary Pruning.
 When we see the plot we see that there are some leaf nodes where we have same samples in multiple classes. Due to random
 selection it might have wrongly predicted due to which accuracy drops'''

###  Cross Validation-Comparing Logistic,KNN,SVM algorithms

##### SVM

In [29]:
from sklearn import svm
classifier=svm.SVC(kernel="rbf", gamma=0.1, C=1)
classifier.fit(X_train,Y_train)
Y_pred=classifier.predict(X_test)

In [30]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cfm=confusion_matrix(Y_test,Y_pred)
print(cfm)

print()

print("classification report:")
print(classification_report(Y_test,Y_pred))

print()

acc=accuracy_score(Y_test, Y_pred)
print("Accuracy of the model: ",acc)

[[ 82   0  20   0]
 [ 21   0   0   0]
 [ 17   0 354   0]
 [ 17   0   0   8]]

classification report:
              precision    recall  f1-score   support

           0       0.60      0.80      0.69       102
           1       0.00      0.00      0.00        21
           2       0.95      0.95      0.95       371
           3       1.00      0.32      0.48        25

    accuracy                           0.86       519
   macro avg       0.64      0.52      0.53       519
weighted avg       0.84      0.86      0.84       519


Accuracy of the model:  0.8554913294797688


C:\Users\Neeharika\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


##### Tuned SVM

In [33]:
from sklearn import svm
classifier=svm.SVC(kernel="rbf", gamma=0.1, C=50)
classifier.fit(X_train,Y_train)
Y_pred=classifier.predict(X_test)

In [34]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cfm=confusion_matrix(Y_test,Y_pred)
print(cfm)

print()

print("classification report:")
print(classification_report(Y_test,Y_pred))

print()

acc=accuracy_score(Y_test, Y_pred)
print("Accuracy of the model: ",acc)

[[100   1   1   0]
 [  0  21   0   0]
 [  0   0 371   0]
 [  0   0   0  25]]

classification report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       102
           1       0.95      1.00      0.98        21
           2       1.00      1.00      1.00       371
           3       1.00      1.00      1.00        25

    accuracy                           1.00       519
   macro avg       0.99      1.00      0.99       519
weighted avg       1.00      1.00      1.00       519


Accuracy of the model:  0.9961464354527938


##### KNN

In [31]:
from sklearn.neighbors import KNeighborsClassifier
model_KNN=KNeighborsClassifier(n_neighbors=8,metric='euclidean')
model_KNN.fit(X_train,Y_train)
Y_pred=model_KNN.predict(X_test)

In [32]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cfm=confusion_matrix(Y_test,Y_pred)
print(cfm)

print()

print("classification report:")
print(classification_report(Y_test,Y_pred))

print()

acc=accuracy_score(Y_test, Y_pred)
print("Accuracy of the model: ",acc)

[[ 97   1   4   0]
 [  8  12   0   1]
 [ 10   0 361   0]
 [  3   0   0  22]]

classification report:
              precision    recall  f1-score   support

           0       0.82      0.95      0.88       102
           1       0.92      0.57      0.71        21
           2       0.99      0.97      0.98       371
           3       0.96      0.88      0.92        25

    accuracy                           0.95       519
   macro avg       0.92      0.84      0.87       519
weighted avg       0.95      0.95      0.95       519


Accuracy of the model:  0.9479768786127167


##### Logistic Regression

In [35]:
from sklearn.linear_model import LogisticRegression
classifier=LogisticRegression()
classifier.fit(X_train,Y_train)
Y_pred=classifier.predict(X_test)

C:\Users\Neeharika\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Neeharika\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [36]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cfm=confusion_matrix(Y_test,Y_pred)
print(cfm)

print()

print("classification report:")
print(classification_report(Y_test,Y_pred))

print()

acc=accuracy_score(Y_test, Y_pred)
print("Accuracy of the model: ",acc)

[[ 22   0  80   0]
 [  3   0  18   0]
 [ 30   0 341   0]
 [ 11   0  14   0]]

classification report:
              precision    recall  f1-score   support

           0       0.33      0.22      0.26       102
           1       0.00      0.00      0.00        21
           2       0.75      0.92      0.83       371
           3       0.00      0.00      0.00        25

    accuracy                           0.70       519
   macro avg       0.27      0.28      0.27       519
weighted avg       0.60      0.70      0.64       519


Accuracy of the model:  0.6994219653179191


C:\Users\Neeharika\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
'''By above cross validation we observe that Tuned SVM has given highest accuracy followed by Base DT and KNN. Logistic is 
not suitable for multiclass predictions. We can take a call among Tuned SVM and Base DT considering complexity and accuracy 
of algorithms. Ideally for tuned SVM we need to do trail and error(for changing hyper parameters-refer SVM code file) 
which leads to complexity due to tuning. Regularizationand penalisation cost incur. Hence if we need to choose b/w two 
we go for simple base DT than complex Tuned SVM'''